<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Natural Language Processing (NLP) Review Lab

_Authors: Joseph Nelson (DC)_

---

> **Note: This lab is intended to be done as a walkthrough with the instructor.**

## Introduction


*Adapted from [NLP Crash Course](http://files.meetup.com/7616132/DC-NLP-2013-09%20Charlie%20Greenbacker.pdf) by Charlie Greenbacker, [Introduction to NLP](http://spark-public.s3.amazonaws.com/nlp/slides/intro.pdf) by Dan Jurafsky, Kevin Markham's Data School Curriculum*

### What is NLP?

- Using computers to process (analyze, understand, generate) natural human languages
- Most knowledge created by humans is unstructured text, and we need a way to make sense of it
- Build probabilistic model using data about a language

### What are some of the higher level task areas?

- **Information retrieval**: Find relevant results and similar results
    - [Google](https://www.google.com/)
- **Information extraction**: Structured information from unstructured documents
    - [Events from Gmail](https://support.google.com/calendar/answer/6084018?hl=en)
- **Machine translation**: One language to another
    - [Google Translate](https://translate.google.com/)
- **Text simplification**: Preserve the meaning of text, but simplify the grammar and vocabulary
    - [Rewordify](https://rewordify.com/)
    - [Simple English Wikipedia](https://simple.wikipedia.org/wiki/Main_Page)
- **Predictive text input**: Faster or easier typing
    - [My application](https://justmarkham.shinyapps.io/textprediction/)
    - [A much better application](https://farsite.shinyapps.io/swiftkey-cap/)
- **Sentiment analysis**: Attitude of speaker
    - [Hater News](http://haternews.herokuapp.com/)
- **Automatic summarization**: Extractive or abstractive summarization
    - [autotldr](https://www.reddit.com/r/technology/comments/35brc8/21_million_people_still_use_aol_dialup/cr2zzj0)
- **Natural Language Generation**: Generate text from data
    - [How a computer describes a sports match](http://www.bbc.com/news/technology-34204052)
    - [Publishers withdraw more than 120 gibberish papers](http://www.nature.com/news/publishers-withdraw-more-than-120-gibberish-papers-1.14763)
- **Speech recognition and generation**: Speech-to-text, text-to-speech
    - [Google's Web Speech API demo](https://www.google.com/intl/en/chrome/demos/speech.html)
    - [Vocalware Text-to-Speech demo](https://www.vocalware.com/index/demo)
- **Question answering**: Determine the intent of the question, match query with knowledge base, evaluate hypotheses
    - [How did supercomputer Watson beat Jeopardy champion Ken Jennings?](http://blog.ted.com/how-did-supercomputer-watson-beat-jeopardy-champion-ken-jennings-experts-discuss/)
    - [IBM's Watson Trivia Challenge](http://www.nytimes.com/interactive/2010/06/16/magazine/watson-trivia-game.html)
    - [The AI Behind Watson](http://www.aaai.org/Magazine/Watson/watson.php)

### What are some of the lower level components?

- **Tokenization**: breaking text into tokens (words, sentences, n-grams)
- **Stopword removal**: a/an/the
- **Stemming and lemmatization**: root word
- **TF-IDF**: word importance
- **Part-of-speech tagging**: noun/verb/adjective
- **Named entity recognition**: person/organization/location
- **Spelling correction**: "New Yrok City"
- **Word sense disambiguation**: "buy a mouse"
- **Segmentation**: "New York City subway"
- **Language detection**: "translate this page"
- **Machine learning**

### Why is NLP hard?

- **Ambiguity**:
    - Hospitals are Sued by 7 Foot Doctors
    - Juvenile Court to Try Shooting Defendant
    - Local High School Dropouts Cut in Half
- **Non-standard English**: text messages
- **Idioms**: "throw in the towel"
- **Newly coined words**: "retweet"
- **Tricky entity names**: "Where is A Bug's Life playing?"
- **World knowledge**: "Mary and Sue are sisters", "Mary and Sue are mothers"

NLP requires an understanding of the **language** and the **world**.

## Part 1: Reading in the Yelp Reviews

- "corpus" = collection of documents
- "corpora" = plural form of corpus

In [37]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

In [38]:
csv_file = '../datasets/yelp.csv'

In [39]:
# A:
yelp = pd.read_csv(csv_file)
yelp.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


### 1.1 Subset the reviews to best and worst.

- Select only 5-star and 1-star reviews.
- The text will be the features, the stars will be the target.
- Create a train-test split.

In [40]:
# A:
yelp_best_and_worst = yelp[(yelp.stars == 5) | (yelp.stars == 1)]
yelp_best_and_worst.stars.value_counts()

5    3337
1     749
Name: stars, dtype: int64

In [41]:
X_train, X_test, y_train, y_test = train_test_split(yelp_best_and_worst.text, yelp_best_and_worst.stars, random_state=1)

## Part 2: Tokenization

- **What:** Separate text into units such as sentences or words
- **Why:** Gives structure to previously unstructured text
- **Notes:** Relatively easy with English language text, not easy with some languages

### 2.1 Use CountVectorizer to convert the training and testing text data.

[CountVectorizer documentation](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

- **lowercase:** boolean, True by default
    - Convert all characters to lowercase before tokenizing.
- **ngram_range:** tuple (min_n, max_n)
    - The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used.

In [42]:
# A:
# initialize the vectorizer
vect = CountVectorizer(ngram_range=(1,2))

# fit the vectorizer and create a vectorized training set
X_train_dtm = vect.fit_transform(X_train)

# vectorize the test data using the fitted vectorizer
X_test_dtm = vect.transform(X_test)

In [43]:
X_train_dtm.shape

(3064, 169847)

### 2.2 Predict the star rating with the new features from CountVectorizer.

Validate on the test set.

In [44]:
# A:
# initialized the Naive Bayes model
nb = MultinomialNB()

# fit the model on the vectorized training data
nb.fit(X_train_dtm, y_train)

# create predictions using the vectorized test data
y_pred = nb.predict(X_test_dtm)

# calculate accuracy of predictions
metrics.accuracy_score(y_test, y_pred)

0.85420743639921726

## Part 3: Stopword Removal

- **What:** Remove common words that will likely appear in any text
- **Why:** They don't tell you much about your text

### 3.1 Recreate your features with CountVectorizer removing stopwords.

- **stop_words:** string {'english'}, list, or None (default)
- If 'english', a built-in stop word list for English is used.
- If a list, that list is assumed to contain stop words, all of which will be removed from the resulting tokens.
- If None, no stop words will be used. max_df can be set to a value in the range [0.7, 1.0) to automatically detect and filter stop words based on intra corpus document frequency of terms.

In [23]:
# A:
# initialize the vectorizer
vect = CountVectorizer(stop_words='english')

# fit the vectorizer on training data and create vectorized training set
X_train_dtm = vect.fit_transform(X_train)

# vectorize the test set
X_test_dtm = vect.transform(X_test)

# fit the NB model
nb.fit(X_train_dtm, y_train)

# predict on test set
y_pred = nb.predict(X_test_dtm)

# score
metrics.accuracy_score(y_test, y_pred)

0.92074363992172215

### 3.2 Validate your model using the features with stopwords removed.

In [8]:
# A: see above

## Part 4: Other CountVectorizer Options

### 4.1 Shrink the maximum number of features and re-test the model.

- **max_features:** int or None, default=None
- If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

In [25]:
# A:
# initialize the vectorizer
vect = CountVectorizer(stop_words='english', max_features=10000)

# fit the vectorizer and vectorize the training data
X_train_dtm = vect.fit_transform(X_train)

# vectorize the testing data
X_test_dtm = vect.transform(X_test)

# fit the model
nb.fit(X_train_dtm, y_train)

# predict on test set
y_pred = nb.predict(X_test_dtm)

# score the predictions
metrics.accuracy_score(y_test, y_pred)

0.92563600782778865

### 4.2 Change the minimum document frequency for terms and test the model's performance.

- **min_df:** float in range [0.0, 1.0] or int, default=1
- When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts.

In [29]:
# A:
# initialize the vectorizer
vect = CountVectorizer(stop_words='english', max_features=10000, min_df=3)

# fit the vectorizer and vectorize the training data
X_train_dtm = vect.fit_transform(X_train)

# vectorize the testing data
X_test_dtm = vect.transform(X_test)

# fit the model
nb.fit(X_train_dtm, y_train)

# predict on test set
y_pred = nb.predict(X_test_dtm)

# score the predictions
metrics.accuracy_score(y_test, y_pred)

0.93052837573385516

## Part 5: Introduction to TextBlob

TextBlob: "Simplified Text Processing"

### 5.1 Use `TextBlob` to convert the text in the first review in the dataset.

In [34]:
# A:
review = TextBlob(yelp.text[0])
review

TextBlob("My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!")

### 5.2 List the words in the `TextBlob` object.

In [35]:
# A:
review.words

WordList(['My', 'wife', 'took', 'me', 'here', 'on', 'my', 'birthday', 'for', 'breakfast', 'and', 'it', 'was', 'excellent', 'The', 'weather', 'was', 'perfect', 'which', 'made', 'sitting', 'outside', 'overlooking', 'their', 'grounds', 'an', 'absolute', 'pleasure', 'Our', 'waitress', 'was', 'excellent', 'and', 'our', 'food', 'arrived', 'quickly', 'on', 'the', 'semi-busy', 'Saturday', 'morning', 'It', 'looked', 'like', 'the', 'place', 'fills', 'up', 'pretty', 'quickly', 'so', 'the', 'earlier', 'you', 'get', 'here', 'the', 'better', 'Do', 'yourself', 'a', 'favor', 'and', 'get', 'their', 'Bloody', 'Mary', 'It', 'was', 'phenomenal', 'and', 'simply', 'the', 'best', 'I', "'ve", 'ever', 'had', 'I', "'m", 'pretty', 'sure', 'they', 'only', 'use', 'ingredients', 'from', 'their', 'garden', 'and', 'blend', 'them', 'fresh', 'when', 'you', 'order', 'it', 'It', 'was', 'amazing', 'While', 'EVERYTHING', 'on', 'the', 'menu', 'looks', 'excellent', 'I', 'had', 'the', 'white', 'truffle', 'scrambled', 'eggs', 

### 5.3 List the sentences in the `TextBlob` object.

In [36]:
# A:
review.sentences

[Sentence("My wife took me here on my birthday for breakfast and it was excellent."),
 Sentence("The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure."),
 Sentence("Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning."),
 Sentence("It looked like the place fills up pretty quickly so the earlier you get here the better."),
 Sentence("Do yourself a favor and get their Bloody Mary."),
 Sentence("It was phenomenal and simply the best I've ever had."),
 Sentence("I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it."),
 Sentence("It was amazing."),
 Sentence("While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious."),
 Sentence("It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete."),
 Sentence("It was the best "toast" I've ever had."),


## Part 6: Stemming and Lemmatization

**Stemming:**

- **What:** Reduce a word to its base/stem/root form
- **Why:** Often makes sense to treat related words the same way
- **Notes:**
    - Uses a "simple" and fast rule-based approach
    - Stemmed words are usually not shown to users (used for analysis/indexing)
    - Some search engines treat words with the same stem as synonyms

### 6.1 Initialize the `SnowballStemmer` and stem the words in the first review.

In [14]:
# A:
SnowballStemmer 

### 6.2 Use the built-in `lemmatize` function on the words of the first review (parsed by `TextBlob`)

**Lemmatization**

- **What:** Derive the canonical form ('lemma') of a word
- **Why:** Can be better than stemming
- **Notes:** Uses a dictionary-based approach (slower than stemming)

In [15]:
# A:

### 6.3 Write a function that uses `TextBlob` and `lemmatize` to lemmatize text.

In [16]:
# A:

### 6.4 Provide your function to `CountVectorizer` as the `analyzer` and test the performance of your model.

In [17]:
# A:

## Part 7: Term Frequency-Inverse Document Frequency (TF-IDF)

- **What:** Computes "relative frequency" that a word appears in a document compared to its frequency across all documents
- **Why:** More useful than "term frequency" for identifying "important" words in each document (high frequency in that document, low frequency in other documents)
- **Notes:** Used for search engine scoring, text summarization, document clustering

### 7.1 Build a simple TF-IDF using CountVectorizer

- Term Frequency can be calulated with default CountVectorizer.
- Inverse Document Frequency can be calculated with CountVectorizer and argument `binary=True`.

**More details:** [TF-IDF is about what matters](http://planspace.org/20150524-tfidf_is_about_what_matters/)

In [18]:
# A:

## Part 8: Using TF-IDF to Summarize a Yelp Review

> **Note:** Reddit's autotldr uses the [SMMRY](http://smmry.com/about) algorithm, which is based on TF-IDF!

### 8.1 Build a TF-IDF predictor matrix excluding stopwords with `TfidfVectorizer`

In [19]:
# A:

### 8.2 Write a function to pull out the top 5 words by TF-IDF score from a review

In [20]:
# A:

## Part 9: Sentiment Analysis

### 9.1 Extract sentiment from a review parsed with `TextBlob`

Sentiment polarity ranges from -1, the most negative, to 1, the most positive. A parsed TextBlob object has sentiment which can be accessed with:

    review.sentiment.polarity

In [21]:
# A:

### 9.2 Calculate the sentiment for every review in the full Yelp dataset as a new column.

In [22]:
# A:

### 9.3 Create a boxplot of sentiment by star rating

In [23]:
# A:

### 9.4 Print reviews with the highest and lowest sentiment.

In [24]:
# A:

## 10. [Bonus] Explore fun TextBlob features

### 10.1 Correct spelling with `.correct()`

In [25]:
# A:

### 10.2 Perform spellchecking with `.spellcheck()`

In [26]:
# A:

### 10.3 Extract definitions with `.define()`

In [27]:
# A:

## Conclusion

- NLP is a gigantic field
- Understanding the basics broadens the types of data you can work with
- Simple techniques go a long way
- Use scikit-learn for NLP whenever possible